In [1]:
import os
import geopandas as gpd
import pandas as pd
import pyarrow as pa
from shapely import wkb
import altair as alt

In [2]:
price_df = pd.read_csv("../data/raw/pricePerProvince.csv", encoding='latin1').sort_values(by='price', ascending=False)
alt_data = pd.read_parquet('../data/processed/kWh_poly.parquet')
alt_data['geometry'] = alt_data['geometry'].apply(wkb.loads)
alt_data = gpd.GeoDataFrame(alt_data)
gdf_ca = gpd.read_parquet('../data/processed/ne_50m_admin_1_states_provinces.parquet')
panel_df = pd.read_csv('../data/raw/panels.csv')

In [3]:
province = "British Columbia"
region = "Vancouver"

In [4]:
province_price = price_df[(price_df['province'] == province)]["price"].iloc[0] / 100
filtered_row = alt_data[(alt_data['Province'] == province) & (alt_data['Municipality'] == region) & (alt_data['Month'] == 'Annual')]

In [5]:
panel_df.head()

,name,full name,efficiency,price,length (mm),width (mm)
0,Canadian Solar 340W (309 CAD),Canadian Solar 340W Solar Panel | CS1H-340MS,0.189,309.0,1701.8,1016.0
1,Canadian Solar 445W (389 CAD),Canadian Solar Bi-facial 445W Solar Panel | CS...,0.189,389.0,2133.6,1066.8
2,Elios Voltaic 210W (299 CAD),Elios Voltaic 210F-B | 210W Semi-Flexible Balc...,0.193,299.0,1083.0,1104.0
3,Elios Voltaic 210W (350 CAD),Elios Voltaic 210F-T | 210W Semi-Flexible Sola...,0.193,350.0,1083.0,1040.0
4,Elois Voltaic 110W (139 CAD),Elios Voltaic110F | 110W Semi-Flexible Solar P...,0.193,139.0,1079.5,562.0


In [6]:
conversion_rate = {row['name ']: row['efficiency '] for index, row in panel_df.iterrows()}

In [7]:
conversion_rate

{'Canadian Solar 340W (309 CAD)': 0.189,
 'Canadian Solar 445W (389 CAD)': 0.189,
 'Elios Voltaic 210W (299 CAD)': 0.193,
 'Elios Voltaic 210W (350 CAD)': 0.193,
 'Elois Voltaic 110W (139 CAD)': 0.193,
 'Elios Voltaic 200W (275 CAD)': 0.229,
 'Elios Voltaic 400W (225 CAD)': 0.205,
 'Elios Voltaic 550W (319 CAD)': 0.213,
 'Hanwha 400W (385 CAD)': 0.196,
 'Hanwha 470W (424 CAD)': 0.203,
 'LG 370W (407 CAD)': 0.193,
 'LG 380W (418 CAD)': 0.206,
 'LG 410W (451 CAD) ': 0.209,
 'LG 445W (490 CAD)': 0.202,
 'Peimar 385W (370 CAD)': 0.194,
 'Peimar 400W (339 CAD)': 0.196,
 'SunPower 100W (199 CAD) ': 0.23,
 'SunPower 50W (135 CAD) ': 0.23,
 'PERC Tech 450W (399 CAD) ': 0.202,
 'PERC Tech 450W (275 CAD) ': 0.196}

In [8]:
conversion_df = pd.DataFrame(list(conversion_rate.items()), columns=['name', 'efficiency'])
####

In [9]:
panel_comparison = ['Canadian Solar 340W (309 CAD)', 'Canadian Solar 445W (389 CAD)']

In [10]:
conversion_df['highlight'] = conversion_df['name'].isin(panel_comparison)
####

In [11]:
comparison_values = [conversion_rate[value] for value in panel_comparison]

In [12]:
num_pan = 5

In [13]:
comparison_savings = []
for value in comparison_values:
    comparison_savings.append(filtered_row['South-facing with vertical (90 degrees) tilt'].iloc[0] * value * 1.65 * 365 * num_pan * province_price)
df = pd.DataFrame({comp: [saving] for comp, saving in zip(panel_comparison, comparison_savings)})
df = df.T.reset_index().rename(columns={'index': 'Panel Comparison', 0: 'Savings'})

In [14]:
comparison_savings

[169.98662835, 169.98662835]

In [27]:
alt.Chart(conversion_df).mark_bar().encode(
    y=alt.Y('name:N', title='Panel Name', sort='-x'), 
    x=alt.X('efficiency:Q', title='Efficiency', scale=alt.Scale(domain=(0, 0.25))),  
    color=alt.Color('highlight:N', scale=alt.Scale(domain=[True, False], range=['gold', 'steelblue']), legend=None),  
    tooltip=['name', 'efficiency']  
).properties(
    title='Panel Efficiency Comparison'
).interactive()


alt.Chart(...)